In [1]:
import os
import pandas as pd
import duckdb

import requests
from io import StringIO

import ibis
import ibis.selectors as s
from ibis import _
ibis.options.interactive = True

from neuralforecast import NeuralForecast
from neuralforecast.models import TSMixerx, TiDE
from neuralforecast.auto import AutoTSMixerx, AutoTiDE

# logging
import logging

# define log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [2]:
os.chdir('../..')

In [3]:
# connect to database
con = ibis.duckdb.connect("data/spp.ddb")
con.list_tables()

['lmp', 'mtlf', 'mtrf']

In [111]:
lmp = con.table('lmp')
lmp

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Interval_HE         ┃ GMTIntervalEnd_HE   ┃ timestamp_mst_HE    ┃ Settlement_Location_Name ┃ PNODE_Name ┃ LMP      ┃ MLC      ┃ MCC       ┃ MEC      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ timestamp(6)        │ !timestamp(6)       │ timestamp(6)        │ !string                  │ !string    │ float64  │ float64  │ float64   │ float64  │
├─────────────────────┼─────────────────────┼─────────────────────┼──────────────────────────┼────────────┼──────────┼──────────┼───────────┼──────────┤
│ 2023-05-02 01:00:00 │ 2023-05-02 06:00:00 │ 2023-05-01 23:00:00 │ AESO                     │ AESO       │ 0.957908 │ 0.018525 │ -0.114242 │ 1.053625 │
│ 2023-05-02 01:00:00 │ 2023-05-02 06:00:00 │ 2023-05-01 23:00:00 │ AVA                      │ AVA        │ 0.958883 │ 0.019467 │ -0.114242 │ 1.053658 │
│ 2023-05-02 01:00:00 │ 2023-05-02 06:00:00 │ 2023-05-01 23:00:00 │ AVRN                     │ AVRN       │ 0.961042 │ 0.021650 │ -0.114242 │ 1.053633 │
│ 2023-05-02 01:00:00 │ 2023-05-02 06:00:00 │ 2023-05-01 23:00:00 │ AZPS                     │ APS        │ 0.975308 │ 0.035892 │ -0.114242 │ 1.053658 │
│ 2023-05-02 01:00:00 │ 2023-05-02 06:00:00 │ 2023-05-01 23:00:00 │ BANC                     │ BANC       │ 0.966342 │ 0.026958 │ -0.114242 │ 1.053625 │
│ 2023-05-02 01:00:00 │ 2023-05-02 06:00:00 │ 2023-05-01 23:00:00 │ BCHA                     │ BCHA       │ 0.959783 │ 0.020392 │ -0.114242 │ 1.053633 │
│ 2023-05-02 01:00:00 │ 2023-05-02 06:00:00 │ 2023-05-01 23:00:00 │ BPA                      │ BPA        │ 0.960842 │ 0.021442 │ -0.114242 │ 1.053642 │
│ 2023-05-02 01:00:00 │ 2023-05-02 06:00:00 │ 2023-05-01 23:00:00 │ CEN                      │ CEN        │ 0.973400 │ 0.034000 │ -0.114242 │ 1.053642 │
│ 2023-05-02 01:00:00 │ 2023-05-02 06:00:00 │ 2023-05-01 23:00:00 │ CHPD                     │ CHPD       │ 0.960283 │ 0.020883 │ -0.114242 │ 1.053642 │
│ 2023-05-02 01:00:00 │ 2023-05-02 06:00:00 │ 2023-05-01 23:00:00 │ CISO                     │ CISO       │ 0.968575 │ 0.029175 │ -0.114242 │ 1.053642 │
│ …                   │ …                   │ …                   │ …                        │ …          │        … │        … │         … │        … │
└─────────────────────┴─────────────────────┴─────────────────────┴──────────────────────────┴────────────┴──────────┴──────────┴───────────┴──────────┘

In [112]:
lmp = lmp.filter(_.Settlement_Location_Name.contains('PSCO'))

In [113]:
lmp.to_pandas()[['GMTIntervalEnd_HE', 'Settlement_Location_Name']].duplicated().sum()

0

In [115]:
drop_cols = [
    'Interval_HE', 'GMTIntervalEnd_HE', 'timestamp_mst_HE',
    'Settlement_Location_Name', 'PNODE_Name', 
    'LMP', 'MLC', 'MCC', 'MEC'
]

lmp = (
    lmp
    .mutate(unique_id = _.Settlement_Location_Name )
    .mutate(ds = _.timestamp_mst_HE)
    .mutate(y = _.LMP) 
    .drop(drop_cols) 
    .order_by(['unique_id', 'ds'])
)

lmp

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ unique_id           ┃ ds                  ┃ y         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ !string             │ timestamp(6)        │ float64   │
├─────────────────────┼─────────────────────┼───────────┤
│ PSCO.BHCE.AIRPORTDL │ 2023-05-01 23:00:00 │  0.900342 │
│ PSCO.BHCE.AIRPORTDL │ 2023-05-02 00:00:00 │  0.904992 │
│ PSCO.BHCE.AIRPORTDL │ 2023-05-02 01:00:00 │  2.601358 │
│ PSCO.BHCE.AIRPORTDL │ 2023-05-02 02:00:00 │ 13.136867 │
│ PSCO.BHCE.AIRPORTDL │ 2023-05-02 03:00:00 │ 10.744717 │
│ PSCO.BHCE.AIRPORTDL │ 2023-05-02 04:00:00 │ 13.265067 │
│ PSCO.BHCE.AIRPORTDL │ 2023-05-02 05:00:00 │ 13.555108 │
│ PSCO.BHCE.AIRPORTDL │ 2023-05-02 06:00:00 │ 15.721517 │
│ PSCO.BHCE.AIRPORTDL │ 2023-05-02 07:00:00 │ 17.135467 │
│ PSCO.BHCE.AIRPORTDL │ 2023-05-02 08:00:00 │ 13.175133 │
│ …                   │ …                   │         … │
└─────────────────────┴─────────────────────┴───────────┘

In [116]:
mtrf = con.table('mtrf')
mtrf

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Interval            ┃ GMTIntervalEnd      ┃ timestamp_mst       ┃ Wind_Forecast_MW ┃ Solar_Forecast_MW ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ timestamp(6)        │ !timestamp(6)       │ timestamp(6)        │ float64          │ float64           │
├─────────────────────┼─────────────────────┼─────────────────────┼──────────────────┼───────────────────┤
│ 2023-05-01 01:00:00 │ 2023-05-01 06:00:00 │ 2023-04-30 23:00:00 │          2085.52 │              0.00 │
│ 2023-05-01 02:00:00 │ 2023-05-01 07:00:00 │ 2023-05-01 00:00:00 │          2549.47 │              0.00 │
│ 2023-05-01 03:00:00 │ 2023-05-01 08:00:00 │ 2023-05-01 01:00:00 │          3017.06 │              0.00 │
│ 2023-05-01 04:00:00 │ 2023-05-01 09:00:00 │ 2023-05-01 02:00:00 │          3213.45 │              0.00 │
│ 2023-05-01 05:00:00 │ 2023-05-01 10:00:00 │ 2023-05-01 03:00:00 │          2723.32 │              0.00 │
│ 2023-05-01 06:00:00 │ 2023-05-01 11:00:00 │ 2023-05-01 04:00:00 │          2574.19 │              0.00 │
│ 2023-05-01 07:00:00 │ 2023-05-01 12:00:00 │ 2023-05-01 05:00:00 │          2499.79 │              0.62 │
│ 2023-05-01 08:00:00 │ 2023-05-01 13:00:00 │ 2023-05-01 06:00:00 │          2070.59 │             55.57 │
│ 2023-05-01 09:00:00 │ 2023-05-01 14:00:00 │ 2023-05-01 07:00:00 │          1672.67 │            259.53 │
│ 2023-05-01 10:00:00 │ 2023-05-01 15:00:00 │ 2023-05-01 08:00:00 │           956.07 │           1015.92 │
│ …                   │ …                   │ …                   │                … │                 … │
└─────────────────────┴─────────────────────┴─────────────────────┴──────────────────┴───────────────────┘

In [117]:
drop_cols = ['Interval', 'GMTIntervalEnd', 'timestamp_mst',]

mtrf = (
    mtrf
    .mutate(ds = _.timestamp_mst)
    .drop(drop_cols) 
    .order_by(['ds'])
)

mtrf

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Wind_Forecast_MW ┃ Solar_Forecast_MW ┃ ds                  ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ float64          │ float64           │ timestamp(6)        │
├──────────────────┼───────────────────┼─────────────────────┤
│          2085.52 │              0.00 │ 2023-04-30 23:00:00 │
│          2549.47 │              0.00 │ 2023-05-01 00:00:00 │
│          3017.06 │              0.00 │ 2023-05-01 01:00:00 │
│          3213.45 │              0.00 │ 2023-05-01 02:00:00 │
│          2723.32 │              0.00 │ 2023-05-01 03:00:00 │
│          2574.19 │              0.00 │ 2023-05-01 04:00:00 │
│          2499.79 │              0.62 │ 2023-05-01 05:00:00 │
│          2070.59 │             55.57 │ 2023-05-01 06:00:00 │
│          1672.67 │            259.53 │ 2023-05-01 07:00:00 │
│           956.07 │           1015.92 │ 2023-05-01 08:00:00 │
│                … │                 … │ …                   │
└──────────────────┴───────────────────┴─────────────────────┘

In [118]:
mtlf = con.table('mtlf')
mtlf

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Interval            ┃ GMTIntervalEnd      ┃ timestamp_mst       ┃ MTLF  ┃ Averaged_Actual ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ timestamp(6)        │ !timestamp(6)       │ timestamp(6)        │ int32 │ float64         │
├─────────────────────┼─────────────────────┼─────────────────────┼───────┼─────────────────┤
│ 2023-05-01 00:00:00 │ 2023-05-01 05:00:00 │ 2023-04-30 22:00:00 │  7209 │          6972.0 │
│ 2023-05-01 01:00:00 │ 2023-05-01 06:00:00 │ 2023-04-30 23:00:00 │  6790 │          6620.0 │
│ 2023-05-01 02:00:00 │ 2023-05-01 07:00:00 │ 2023-05-01 00:00:00 │  6477 │          6384.0 │
│ 2023-05-01 03:00:00 │ 2023-05-01 08:00:00 │ 2023-05-01 01:00:00 │  6319 │          6226.0 │
│ 2023-05-01 04:00:00 │ 2023-05-01 09:00:00 │ 2023-05-01 02:00:00 │  6278 │          6316.0 │
│ 2023-05-01 05:00:00 │ 2023-05-01 10:00:00 │ 2023-05-01 03:00:00 │  6282 │          6440.0 │
│ 2023-05-01 06:00:00 │ 2023-05-01 11:00:00 │ 2023-05-01 04:00:00 │  6430 │          6575.0 │
│ 2023-05-01 07:00:00 │ 2023-05-01 12:00:00 │ 2023-05-01 05:00:00 │  6980 │          6879.0 │
│ 2023-05-01 08:00:00 │ 2023-05-01 13:00:00 │ 2023-05-01 06:00:00 │  7399 │          7169.0 │
│ 2023-05-01 09:00:00 │ 2023-05-01 14:00:00 │ 2023-05-01 07:00:00 │  7398 │          7252.0 │
│ …                   │ …                   │ …                   │     … │               … │
└─────────────────────┴─────────────────────┴─────────────────────┴───────┴─────────────────┘

In [119]:
drop_cols = ['Interval', 'GMTIntervalEnd', 'timestamp_mst',]

mtlf = (
    mtlf
    .mutate(ds = _.timestamp_mst)
    .drop(drop_cols) 
    .order_by(['ds'])
)

mtlf

┏━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ MTLF  ┃ Averaged_Actual ┃ ds                  ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int32 │ float64         │ timestamp(6)        │
├───────┼─────────────────┼─────────────────────┤
│  7209 │          6972.0 │ 2023-04-30 22:00:00 │
│  6790 │          6620.0 │ 2023-04-30 23:00:00 │
│  6477 │          6384.0 │ 2023-05-01 00:00:00 │
│  6319 │          6226.0 │ 2023-05-01 01:00:00 │
│  6278 │          6316.0 │ 2023-05-01 02:00:00 │
│  6282 │          6440.0 │ 2023-05-01 03:00:00 │
│  6430 │          6575.0 │ 2023-05-01 04:00:00 │
│  6980 │          6879.0 │ 2023-05-01 05:00:00 │
│  7399 │          7169.0 │ 2023-05-01 06:00:00 │
│  7398 │          7252.0 │ 2023-05-01 07:00:00 │
│     … │               … │ …                   │
└───────┴─────────────────┴─────────────────────┘

In [120]:
all_df = (
    mtlf
    .left_join(mtrf, 'ds')
    .select(~s.contains("_right")) # remove 'dt_right'
    .left_join(lmp, 'ds')
    .select(~s.contains("_right")) # remove 'dt_right'
    .order_by(['unique_id', 'ds'])
)
all_df 


┏━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ MTLF  ┃ Averaged_Actual ┃ ds                  ┃ Wind_Forecast_MW ┃ Solar_Forecast_MW ┃ unique_id           ┃ y         ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ int32 │ float64         │ timestamp(6)        │ float64          │ float64           │ string              │ float64   │
├───────┼─────────────────┼─────────────────────┼──────────────────┼───────────────────┼─────────────────────┼───────────┤
│  6818 │          6809.0 │ 2023-05-01 23:00:00 │          3571.73 │              0.00 │ PSCO.BHCE.AIRPORTDL │  0.900342 │
│  6597 │          6511.0 │ 2023-05-02 00:00:00 │          3339.18 │              0.00 │ PSCO.BHCE.AIRPORTDL │  0.904992 │
│  6507 │          6372.0 │ 2023-05-02 01:00:00 │          3195.45 │              0.00 │ PSCO.BHCE.AIRPORTDL │  2.601358 │
│  6344 │          6334.0 │ 2023-05-02 02:00:00 │          3296.07 │              0.00 │ PSCO.BHCE.AIRPORTDL │ 13.136867 │
│  6384 │          6362.0 │ 2023-05-02 03:00:00 │          2997.68 │              0.00 │ PSCO.BHCE.AIRPORTDL │ 10.744717 │
│  6550 │          6466.0 │ 2023-05-02 04:00:00 │          3080.94 │              0.00 │ PSCO.BHCE.AIRPORTDL │ 13.265067 │
│  6903 │          6723.0 │ 2023-05-02 05:00:00 │          2774.13 │              0.54 │ PSCO.BHCE.AIRPORTDL │ 13.555108 │
│  7252 │          7095.0 │ 2023-05-02 06:00:00 │          2799.46 │             49.52 │ PSCO.BHCE.AIRPORTDL │ 15.721517 │
│  7370 │          7236.0 │ 2023-05-02 07:00:00 │          2290.13 │            204.61 │ PSCO.BHCE.AIRPORTDL │ 17.135467 │
│  7270 │          7199.0 │ 2023-05-02 08:00:00 │          2254.61 │            592.32 │ PSCO.BHCE.AIRPORTDL │ 13.175133 │
│     … │               … │ …                   │                … │                 … │ …                   │         … │
└───────┴─────────────────┴─────────────────────┴──────────────────┴───────────────────┴─────────────────────┴───────────┘

In [121]:
all_df.count()

┌─────────┐
│ 1353408 │
└─────────┘

In [122]:
all_df = (
    all_df
    .drop_null(['unique_id'])
    .mutate(hour = _.ds.hour())
    .mutate(weekday = _.ds.day_of_week.index())
    .mutate(month = _.ds.month())
    .mutate(re_ratio = (_.Wind_Forecast_MW + _.Solar_Forecast_MW) / _.MTLF)
)

all_df

┏━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┓
┃ MTLF  ┃ Averaged_Actual ┃ ds                  ┃ Wind_Forecast_MW ┃ Solar_Forecast_MW ┃ unique_id           ┃ y         ┃ hour  ┃ weekday ┃ month ┃ re_ratio ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━┩
│ int32 │ float64         │ timestamp(6)        │ float64          │ float64           │ string              │ float64   │ int32 │ int16   │ int32 │ float64  │
├───────┼─────────────────┼─────────────────────┼──────────────────┼───────────────────┼─────────────────────┼───────────┼───────┼─────────┼───────┼──────────┤
│  6818 │          6809.0 │ 2023-05-01 23:00:00 │          3571.73 │              0.00 │ PSCO.BHCE.AIRPORTDL │  0.900342 │    23 │       0 │     5 │ 0.523868 │
│  6597 │          6511.0 │ 2023-05-02 00:00:00 │          3339.18 │              0.00 │ PSCO.BHCE.AIRPORTDL │  0.904992 │     0 │       1 │     5 │ 0.506166 │
│  6507 │          6372.0 │ 2023-05-02 01:00:00 │          3195.45 │              0.00 │ PSCO.BHCE.AIRPORTDL │  2.601358 │     1 │       1 │     5 │ 0.491079 │
│  6344 │          6334.0 │ 2023-05-02 02:00:00 │          3296.07 │              0.00 │ PSCO.BHCE.AIRPORTDL │ 13.136867 │     2 │       1 │     5 │ 0.519557 │
│  6384 │          6362.0 │ 2023-05-02 03:00:00 │          2997.68 │              0.00 │ PSCO.BHCE.AIRPORTDL │ 10.744717 │     3 │       1 │     5 │ 0.469561 │
│  6550 │          6466.0 │ 2023-05-02 04:00:00 │          3080.94 │              0.00 │ PSCO.BHCE.AIRPORTDL │ 13.265067 │     4 │       1 │     5 │ 0.470373 │
│  6903 │          6723.0 │ 2023-05-02 05:00:00 │          2774.13 │              0.54 │ PSCO.BHCE.AIRPORTDL │ 13.555108 │     5 │       1 │     5 │ 0.401951 │
│  7252 │          7095.0 │ 2023-05-02 06:00:00 │          2799.46 │             49.52 │ PSCO.BHCE.AIRPORTDL │ 15.721517 │     6 │       1 │     5 │ 0.392854 │
│  7370 │          7236.0 │ 2023-05-02 07:00:00 │          2290.13 │            204.61 │ PSCO.BHCE.AIRPORTDL │ 17.135467 │     7 │       1 │     5 │ 0.338499 │
│  7270 │          7199.0 │ 2023-05-02 08:00:00 │          2254.61 │            592.32 │ PSCO.BHCE.AIRPORTDL │ 13.175133 │     8 │       1 │     5 │ 0.391600 │
│     … │               … │ …                   │                … │                 … │ …                   │         … │     … │       … │     … │        … │
└───────┴─────────────────┴─────────────────────┴──────────────────┴───────────────────┴─────────────────────┴───────────┴───────┴─────────┴───────┴──────────┘

In [123]:
all_df_pd = all_df.to_pandas()
all_df_pd

,MTLF,Averaged_Actual,ds,Wind_Forecast_MW,Solar_Forecast_MW,unique_id,y,hour,weekday,month,re_ratio
0,6818,6809.0,2023-05-01 23:00:00,3571.73,0.00,PSCO.BHCE.AIRPORTDL,0.900342,23,0,5,0.523868
1,6597,6511.0,2023-05-02 00:00:00,3339.18,0.00,PSCO.BHCE.AIRPORTDL,0.904992,0,1,5,0.506166
2,6507,6372.0,2023-05-02 01:00:00,3195.45,0.00,PSCO.BHCE.AIRPORTDL,2.601358,1,1,5,0.491079
3,6344,6334.0,2023-05-02 02:00:00,3296.07,0.00,PSCO.BHCE.AIRPORTDL,13.136867,2,1,5,0.519557
4,6384,6362.0,2023-05-02 03:00:00,2997.68,0.00,PSCO.BHCE.AIRPORTDL,10.744717,3,1,5,0.469561
...,...,...,...,...,...,...,...,...,...,...,...
1353378,10450,10672.0,2024-08-23 19:00:00,2119.33,45.43,PSCO_UPIP_UPI,26.209600,19,4,8,0.207154
1353379,10227,10320.0,2024-08-23 20:00:00,1323.80,0.00,PSCO_UPIP_UPI,19.366533,20,4,8,0.129442
1353380,9848,9768.0,2024-08-23 21:00:00,1528.30,0.00,PSCO_UPIP_UPI,17.024517,21,4,8,0.155189
1353381,9121,9064.0,2024-08-23 22:00:00,1888.73,0.00,PSCO_UPIP_UPI,13.986367,22,4,8,0.207075


In [124]:
all_df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1353383 entries, 0 to 1353382
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   MTLF               1353383 non-null  int32         
 1   Averaged_Actual    1353383 non-null  float64       
 2   ds                 1353383 non-null  datetime64[us]
 3   Wind_Forecast_MW   1353383 non-null  float64       
 4   Solar_Forecast_MW  1353383 non-null  float64       
 5   unique_id          1353383 non-null  object        
 6   y                  1353383 non-null  float64       
 7   hour               1353383 non-null  int32         
 8   weekday            1353383 non-null  int16         
 9   month              1353383 non-null  int32         
 10  re_ratio           1353383 non-null  float64       
dtypes: datetime64[us](1), float64(5), int16(1), int32(3), object(1)
memory usage: 90.3+ MB


In [167]:
len(all_df_pd.ds.unique()) * len(all_df_pd.unique_id.unique())

1451268

In [125]:
node_groups = all_df_pd.unique_id.unique()
log.info(f'number of nodes: {len(node_groups)}')
node_groups

INFO:__main__:number of nodes: 126


array(['PSCO.BHCE.AIRPORTDL', 'PSCO.BHCE.BRWF1', 'PSCO.BHCE.BRWF2',
       'PSCO.BHCE.PAGS1', 'PSCO.BHCE.PAGS2', 'PSCO.BHCE.PAGS4',
       'PSCO.BHCE.PAGS5', 'PSCO.BHCE.PAGS6', 'PSCO.BHCE.PEAKVIEW',
       'PSCO.BHCE.PPLANTDL', 'PSCO.BHCE.RFDIESEL', 'PSCO.CORE.HUNTER.SLR',
       'PSCO.CORE.PIONEER.SLR', 'PSCO.CORE.VICTORY.SLR',
       'PSCO.CSUM.PRPA_SCWIND', 'PSCO.CSUM.PUEBLOHYDRO',
       'PSCO.GUZ.BRONCO', 'PSCO.HCE.BLANCAPK.SLR', 'PSCO.HCE.EAGLEVLY',
       'PSCO.HCE.HUNTER.SLR', 'PSCO.HCE.ORCHARD', 'PSCO.HCGE.BRONCO',
       'PSCO.PRPM_W.CRAIG1', 'PSCO.PRPM_W.CRAIG2', 'PSCO.PRPM_W.FLATS',
       'PSCO.PRPM_W.MEDICINEBOW', 'PSCO.PRPM_W.PRAIRIE',
       'PSCO.PRPM_W.RAWHIDE1', 'PSCO.PRPM_W.RAWHIDEA',
       'PSCO.PRPM_W.RAWHIDEB', 'PSCO.PRPM_W.RAWHIDEC',
       'PSCO.PRPM_W.RAWHIDED', 'PSCO.PRPM_W.RAWHIDEF',
       'PSCO.PRPM_W.ROUNDHOUSE', 'PSCO.PSC.BCP2', 'PSCO.PSCM.ALAMOSA.SLR',
       'PSCO.PSCM.ARAP567.CC', 'PSCO.PSCM.ATERG1', 'PSCO.PSCM.ATERG2',
       'PSCO.PSCM.BIGHORN.SLR'

In [126]:
node_groups = [node for node in node_groups if 'PSCO_' in node]
log.info(f'number of nodes: {len(node_groups)}')
node_groups

INFO:__main__:number of nodes: 9


['PSCO_BHCE',
 'PSCO_CRSP',
 'PSCO_MEAN_W',
 'PSCO_PRPM_PR',
 'PSCO_PSCM_PSCO_PSCM',
 'PSCO_PSCM_PSCP_ARPA',
 'PSCO_PSCO_PSCO_RPSP',
 'PSCO_TSPM_TS',
 'PSCO_UPIP_UPI']

In [169]:
all_df_pd = all_df_pd[all_df_pd.unique_id.isin(node_groups)].reset_index(drop=True)
all_df_pd

,MTLF,Averaged_Actual,ds,Wind_Forecast_MW,Solar_Forecast_MW,unique_id,y,hour,weekday,month,re_ratio
0,6818,6809.0,2023-05-01 23:00:00,3571.73,0.00,PSCO_BHCE,0.922608,23,0,5,0.523868
1,6597,6511.0,2023-05-02 00:00:00,3339.18,0.00,PSCO_BHCE,0.926283,0,1,5,0.506166
2,6507,6372.0,2023-05-02 01:00:00,3195.45,0.00,PSCO_BHCE,2.659375,1,1,5,0.491079
3,6344,6334.0,2023-05-02 02:00:00,3296.07,0.00,PSCO_BHCE,13.408308,2,1,5,0.519557
4,6384,6362.0,2023-05-02 03:00:00,2997.68,0.00,PSCO_BHCE,10.971367,3,1,5,0.469561
...,...,...,...,...,...,...,...,...,...,...,...
85399,10450,10672.0,2024-08-23 19:00:00,2119.33,45.43,PSCO_UPIP_UPI,26.209600,19,4,8,0.207154
85400,10227,10320.0,2024-08-23 20:00:00,1323.80,0.00,PSCO_UPIP_UPI,19.366533,20,4,8,0.129442
85401,9848,9768.0,2024-08-23 21:00:00,1528.30,0.00,PSCO_UPIP_UPI,17.024517,21,4,8,0.155189
85402,9121,9064.0,2024-08-23 22:00:00,1888.73,0.00,PSCO_UPIP_UPI,13.986367,22,4,8,0.207075


In [170]:
horizon = 48
input_size = 14
futr_exog_list = ['MTLF', 'Wind_Forecast_MW', 'Solar_Forecast_MW', 're_ratio', 'hour', 'weekday', 'month']
hist_exog_list = ['Averaged_Actual']
n_series = len(node_groups)

model = TSMixerx(
    h = horizon,
    input_size = input_size,
    n_series = n_series,
    futr_exog_list = futr_exog_list,
    hist_exog_list = hist_exog_list,
    scaler_type = 'robust', 
    max_steps=10_000
)

Seed set to 1


In [171]:
from neuralforecast.core import NeuralForecast

In [172]:
nf = NeuralForecast(models=[model], freq='H')

In [173]:
nf.fit(df=all_df_pd)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type                     | Params | Mode 
-------------------------------------------------------------------------
0 | loss                | MAE                      | 0      | train
1 | padder              | ConstantPad1d            | 0      | train
2 | scaler              | TemporalNorm             | 0      | train
3 | norm                | ReversibleInstanceNorm1d | 18     | train
4 | temporal_projection | Linear                   | 720    | train
5 | feature_mixer_hist  | FeatureMixing            | 20.8 K | train
6 | feature_mixer_futr  | FeatureMixing            | 18.5 K | train
7 | first_mixing        | MixingLayer              | 41.5 K | train
8 | mixing_block        | Sequential               | 45.9 K | train
9 | out                 | Linear                   | 585    | train
-------------------

Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_steps=10000` reached.


In [174]:
# nf.predict?

In [175]:
# futr_df_cols = ['unique_id', 'ds'] + futr_exog_list
# df_cols = ['unique_id', 'ds', 'y'] + hist_exog_list

In [183]:
# pred_idx_df = all_df_pd.ds > '2024-07-13'
pred_idx_futr = all_df_pd.ds <= '2024-07-16'
pred_idx_futr

0         True
1         True
2         True
3         True
4         True
         ...  
85399    False
85400    False
85401    False
85402    False
85403    False
Name: ds, Length: 85404, dtype: bool

In [181]:
pred_idx_futr.sum()

1235447

In [186]:
import numpy as np
fcast_ids = ['PSCO_PRPM_PR']
futr_df = all_df_pd.loc[pred_idx_futr].copy()
futr_df 

,MTLF,Averaged_Actual,ds,Wind_Forecast_MW,Solar_Forecast_MW,unique_id,y,hour,weekday,month,re_ratio
0,6818,6809.0,2023-05-01 23:00:00,3571.73,0.00,PSCO_BHCE,0.922608,23,0,5,0.523868
1,6597,6511.0,2023-05-02 00:00:00,3339.18,0.00,PSCO_BHCE,0.926283,0,1,5,0.506166
2,6507,6372.0,2023-05-02 01:00:00,3195.45,0.00,PSCO_BHCE,2.659375,1,1,5,0.491079
3,6344,6334.0,2023-05-02 02:00:00,3296.07,0.00,PSCO_BHCE,13.408308,2,1,5,0.519557
4,6384,6362.0,2023-05-02 03:00:00,2997.68,0.00,PSCO_BHCE,10.971367,3,1,5,0.469561
...,...,...,...,...,...,...,...,...,...,...,...
84464,10427,10707.0,2024-07-15 20:00:00,2010.21,2.06,PSCO_UPIP_UPI,13.580283,20,0,7,0.192986
84465,10176,10338.0,2024-07-15 21:00:00,2573.59,0.03,PSCO_UPIP_UPI,15.464033,21,0,7,0.252911
84466,9471,9621.0,2024-07-15 22:00:00,2061.36,0.00,PSCO_UPIP_UPI,15.789042,22,0,7,0.217650
84467,8853,8893.0,2024-07-15 23:00:00,1676.70,0.00,PSCO_UPIP_UPI,27.018117,23,0,7,0.189393


In [187]:
pred_idx_df = futr_df.ds > '2024-07-13'
futr_df.loc[pred_idx_df, ['y'] + hist_exog_list] = np.nan
futr_df

,MTLF,Averaged_Actual,ds,Wind_Forecast_MW,Solar_Forecast_MW,unique_id,y,hour,weekday,month,re_ratio
0,6818,6809.0,2023-05-01 23:00:00,3571.73,0.00,PSCO_BHCE,0.922608,23,0,5,0.523868
1,6597,6511.0,2023-05-02 00:00:00,3339.18,0.00,PSCO_BHCE,0.926283,0,1,5,0.506166
2,6507,6372.0,2023-05-02 01:00:00,3195.45,0.00,PSCO_BHCE,2.659375,1,1,5,0.491079
3,6344,6334.0,2023-05-02 02:00:00,3296.07,0.00,PSCO_BHCE,13.408308,2,1,5,0.519557
4,6384,6362.0,2023-05-02 03:00:00,2997.68,0.00,PSCO_BHCE,10.971367,3,1,5,0.469561
...,...,...,...,...,...,...,...,...,...,...,...
84464,10427,NaN,2024-07-15 20:00:00,2010.21,2.06,PSCO_UPIP_UPI,NaN,20,0,7,0.192986
84465,10176,NaN,2024-07-15 21:00:00,2573.59,0.03,PSCO_UPIP_UPI,NaN,21,0,7,0.252911
84466,9471,NaN,2024-07-15 22:00:00,2061.36,0.00,PSCO_UPIP_UPI,NaN,22,0,7,0.217650
84467,8853,NaN,2024-07-15 23:00:00,1676.70,0.00,PSCO_UPIP_UPI,NaN,23,0,7,0.189393


In [188]:
futr_df = futr_df.loc[futr_df.unique_id.isin(fcast_ids)].reset_index(drop=True)
futr_df.sort_values(['ds'])

,MTLF,Averaged_Actual,ds,Wind_Forecast_MW,Solar_Forecast_MW,unique_id,y,hour,weekday,month,re_ratio
0,6818,6809.0,2023-05-01 23:00:00,3571.73,0.00,PSCO_PRPM_PR,0.940458,23,0,5,0.523868
1,6597,6511.0,2023-05-02 00:00:00,3339.18,0.00,PSCO_PRPM_PR,0.941750,0,1,5,0.506166
2,6507,6372.0,2023-05-02 01:00:00,3195.45,0.00,PSCO_PRPM_PR,2.699692,1,1,5,0.491079
3,6344,6334.0,2023-05-02 02:00:00,3296.07,0.00,PSCO_PRPM_PR,13.471892,2,1,5,0.519557
4,6384,6362.0,2023-05-02 03:00:00,2997.68,0.00,PSCO_PRPM_PR,11.063158,3,1,5,0.469561
...,...,...,...,...,...,...,...,...,...,...,...
10578,10427,NaN,2024-07-15 20:00:00,2010.21,2.06,PSCO_PRPM_PR,NaN,20,0,7,0.192986
10579,10176,NaN,2024-07-15 21:00:00,2573.59,0.03,PSCO_PRPM_PR,NaN,21,0,7,0.252911
10580,9471,NaN,2024-07-15 22:00:00,2061.36,0.00,PSCO_PRPM_PR,NaN,22,0,7,0.217650
10581,8853,NaN,2024-07-15 23:00:00,1676.70,0.00,PSCO_PRPM_PR,NaN,23,0,7,0.189393


In [189]:
futr_df.ds.duplicated().sum()

0

In [190]:
nf.predict(
    futr_df=futr_df,
)

/home/justinfields/Documents/spp_weis_price_forecast/venv/lib/python3.11/site-packages/utilsforecast/processing.py:384: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
/home/justinfields/Documents/spp_weis_price_forecast/venv/lib/python3.11/site-packages/utilsforecast/processing.py:438: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  freq = pd.tseries.frequencies.to_offset(freq)


ValueError: There are missing combinations of ids and times in `futr_df`.
You can run the `make_future_dataframe()` method to get the expected combinations or the `get_missing_future(futr_df)` method to get the missing combinations.

In [192]:
nf.get_missing_future(futr_df)

/home/justinfields/Documents/spp_weis_price_forecast/venv/lib/python3.11/site-packages/utilsforecast/processing.py:384: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
/home/justinfields/Documents/spp_weis_price_forecast/venv/lib/python3.11/site-packages/utilsforecast/processing.py:438: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  freq = pd.tseries.frequencies.to_offset(freq)


,unique_id,ds
0,PSCO_BHCE,2024-08-24 00:00:00
1,PSCO_BHCE,2024-08-24 01:00:00
2,PSCO_BHCE,2024-08-24 02:00:00
3,PSCO_BHCE,2024-08-24 03:00:00
4,PSCO_BHCE,2024-08-24 04:00:00
...,...,...
427,PSCO_UPIP_UPI,2024-08-25 19:00:00
428,PSCO_UPIP_UPI,2024-08-25 20:00:00
429,PSCO_UPIP_UPI,2024-08-25 21:00:00
430,PSCO_UPIP_UPI,2024-08-25 22:00:00


In [193]:
nf.make_future_dataframe()

/home/justinfields/Documents/spp_weis_price_forecast/venv/lib/python3.11/site-packages/utilsforecast/processing.py:384: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
/home/justinfields/Documents/spp_weis_price_forecast/venv/lib/python3.11/site-packages/utilsforecast/processing.py:438: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  freq = pd.tseries.frequencies.to_offset(freq)


,unique_id,ds
0,PSCO_BHCE,2024-08-24 00:00:00
1,PSCO_BHCE,2024-08-24 01:00:00
2,PSCO_BHCE,2024-08-24 02:00:00
3,PSCO_BHCE,2024-08-24 03:00:00
4,PSCO_BHCE,2024-08-24 04:00:00
...,...,...
427,PSCO_UPIP_UPI,2024-08-25 19:00:00
428,PSCO_UPIP_UPI,2024-08-25 20:00:00
429,PSCO_UPIP_UPI,2024-08-25 21:00:00
430,PSCO_UPIP_UPI,2024-08-25 22:00:00
